# Preprocessing of claim-sample pairs using a spaCy pipeline.

Ref: 
- [spaCy linguistic features](https://spacy.io/usage/linguistic-features)
- [spaCy processing pipelines](https://spacy.io/usage/processing-pipelines)
- [spaCy custom components](https://spacy.io/usage/processing-pipelines#custom-components)

In [1]:
# Change the working directory to project root
import pathlib
import os
ROOT_DIR = pathlib.Path.cwd()
while not ROOT_DIR.joinpath("src").exists():
    ROOT_DIR = ROOT_DIR.parent
os.chdir(ROOT_DIR)

In [2]:
# Imports and dependencies
import pandas as pd
import numpy as np
import spacy
import torch
from sentence_transformers import SentenceTransformer, util
from src.torch_utils import get_torch_device
from src.spacy_utils import repl_special_token
from typing import Callable, Tuple
import copy
import re

random_seed = 42
np.random.seed(random_seed)
torch_device = get_torch_device()

Torch device is 'mps'


/opt/homebrew/Caskroom/miniconda/base/envs/comp90042_project/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Select models

In [3]:
nlp = spacy.load("en_core_web_trf")
nlp

In [4]:
embedder = SentenceTransformer(
    "sentence-transformers/msmarco-bert-base-dot-v5",
    device=torch_device
)
embedder

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

## Load pair samples

Data need to shape $(n, 5)$ with columns:
- `claim`: claim id
- `claim_text`: claim text string
- `evidence`: evidence id
- `evidence_text`: evidence text string
- `related`: relation labels as `1/0`

In [5]:
train_data_file_path = \
    ROOT_DIR.joinpath("./result/train_data/train_claim_evidence_pair_rns.json")
with open(train_data_file_path, mode="r") as f:
    train_data = (
        pd.read_json(f, orient="records")
        .set_index(["claim", "evidence"])
    )
print(train_data.shape)
train_data.head(60)

(12366, 3)


claim_text   
claim      evidence                                                              
claim-1937 evidence-442946   Not only is there no scientific evidence that ...  \
           evidence-1194317  Not only is there no scientific evidence that ...   
           evidence-12171    Not only is there no scientific evidence that ...   
claim-126  evidence-338219   El Niño drove record highs in global temperatu...   
           evidence-1127398  El Niño drove record highs in global temperatu...   
claim-2510 evidence-530063              In 1946, PDO switched to a cool phase.   
           evidence-984887              In 1946, PDO switched to a cool phase.   
claim-2021 evidence-1177431  Weather Channel co-founder John Coleman provid...   
           evidence-782448   Weather Channel co-founder John Coleman provid...   
           evidence-540069   Weather Channel co-founder John Coleman provid...   
           evidence-352655   Weather Channel co-founder John Coleman provid...   
           evidence-1007867  Weather Channel co-founder John Coleman provid...   
claim-2449 evidence-1010750  "January 2008 capped a 12 month period of glob...   
           evidence-91661    "January 2008 capped a 12 month period of glob...   
           evidence-722725   "January 2008 capped a 12 month period of glob...   
           evidence-554161   "January 2008 capped a 12 month period of glob...   
           evidence-430839   "January 2008 capped a 12 month period of glob...   
claim-851  evidence-226174   The last time the planet was even four degrees...   
           evidence-1049316  The last time the planet was even four degrees...   
           evidence-358301   The last time the planet was even four degrees...   
           evidence-493329   The last time the planet was even four degrees...   
           evidence-610497   The last time the planet was even four degrees...   
claim-2773 evidence-974673   Tree-ring proxy reconstructions are reliable b...   
           evidence-602109   Tree-ring proxy reconstructions are reliable b...   
claim-949  evidence-707654   Under the most ambitious scenarios, they found...   
           evidence-28478    Under the most ambitious scenarios, they found...   
           evidence-491579   Under the most ambitious scenarios, they found...   
claim-1019 evidence-863309   An additional kick was supplied by an El Niño ...   
           evidence-61462    An additional kick was supplied by an El Niño ...   
           evidence-639818   An additional kick was supplied by an El Niño ...   
           evidence-757821   An additional kick was supplied by an El Niño ...   
           evidence-263527   An additional kick was supplied by an El Niño ...   
claim-2834 evidence-439640   When stomata-derived CO2 (red) is compared to ...   
claim-1441 evidence-217743   195 countries signed the 2015 Paris Agreement,...   
claim-1181 evidence-222694   However, there is a process of accretion, wher...   
           evidence-905909   However, there is a process of accretion, wher...   
           evidence-600745   However, there is a process of accretion, wher...   
           evidence-337702   However, there is a process of accretion, wher...   
           evidence-1015241  However, there is a process of accretion, wher...   
claim-2417 evidence-386828   Mars, Triton, Pluto and Jupiter all show globa...   
           evidence-535248   Mars, Triton, Pluto and Jupiter all show globa...   
           evidence-1078011  Mars, Triton, Pluto and Jupiter all show globa...   
           evidence-975483   Mars, Triton, Pluto and Jupiter all show globa...   
           evidence-177166   Mars, Triton, Pluto and Jupiter all show globa...   
claim-2152 evidence-515817      Venus doesn't have a runaway greenhouse effect   
           evidence-1018575     Venus doesn't have a runaway greenhouse effect   
           evidence-791159      Venus doesn't have a runaway greenhouse effect   
           evidence-1009205     Venus doesn't have

## Basic preprocessing exploration

In [6]:
def get_emb_similarity(claim_texts:list, evidence_texts:list) -> Tuple[float]:
    emb_kwargs = {"convert_to_tensor": True, "device": torch_device}
    claim_emb = embedder.encode(sentences=claim_texts, **emb_kwargs)
    evidence_emb = embedder.encode(sentences=evidence_texts, **emb_kwargs)
    score = util.dot_score(a=claim_emb, b=evidence_emb)
    return score

### Lower case

Impact of casting to lower case.

In [7]:
test_A = train_data.loc[("claim-2510", "evidence-984887")]
test_A

/var/folders/j0/q0c6qv3j52qfv9gjdc2q6d1r0000gn/T/com.apple.shortcuts.mac-helper/ipykernel_32780/1524429125.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  test_A = train_data.loc[("claim-2510", "evidence-984887")]


,,claim_text,evidence_text,related
claim,evidence,,,
claim-2510,evidence-984887,"In 1946, PDO switched to a cool phase.","1945/1946: The PDO changed to a ""cool"" phase, ...",1


In [8]:
get_emb_similarity(
    claim_texts=test_A["claim_text"].tolist(),
    evidence_texts=test_A["evidence_text"].values.tolist()
)

tensor([[183.0319]], device='mps:0')

In [9]:
get_emb_similarity(
    claim_texts=test_A["claim_text"].str.lower().tolist(),
    evidence_texts=test_A["evidence_text"].str.lower().values.tolist()
)

tensor([[183.0319]], device='mps:0')

### Replace special tokens

In [11]:
test_nlp = copy.deepcopy(nlp)
test_nlp.add_pipe("repl_special_token", first=True)
test_nlp.pipe_names

['repl_special_token',
 'transformer',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner']

#### Test case: CO2, CO 2, carbon dioxide

In [20]:
test_A = train_data.loc[("claim-1937", "evidence-12171")]
test_A

/var/folders/j0/q0c6qv3j52qfv9gjdc2q6d1r0000gn/T/com.apple.shortcuts.mac-helper/ipykernel_32780/2445213673.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  test_A = train_data.loc[("claim-1937", "evidence-12171")]


,,claim_text,evidence_text,related
claim,evidence,,,
claim-1937,evidence-12171,Not only is there no scientific evidence that ...,Higher carbon dioxide concentrations will favo...,1


In [21]:
get_emb_similarity(
    claim_texts=test_A["claim_text"].tolist(),
    evidence_texts=test_A["evidence_text"].values.tolist()
)

tensor([[171.6822]], device='mps:0')

In [29]:
test_B = test_A.copy()
cols = ["claim_text", "evidence_text"]
test_B[cols] = test_A[cols].applymap(lambda t: test_nlp(t).text)
test_B

,,claim_text,evidence_text,related
claim,evidence,,,
claim-1937,evidence-12171,Not only is there no scientific evidence that ...,Higher carbon dioxide concentrations will favo...,1


In [30]:
get_emb_similarity(
    claim_texts=test_B["claim_text"].tolist(),
    evidence_texts=test_B["evidence_text"].values.tolist()
)

tensor([[175.0602]], device='mps:0')